#### *ISEL - DEI - LEIM*
## Aprendizagem Automática [T52D]
### Trabalho Laboratorial 2: Classificação de Críticas de Cinema do IMDb

João Madeira ($48630$), 
Renata Góis ($51038$),
Bruno Pereira ($51811$)

**Docentes responsáveis:** 
- Prof. Gonçalo Xufre Silva

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as p
import re
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [2]:
with open("resources/imdbFull.p", "rb") as f:
    D = p.load(f)
print("Keys:", D.keys())

reviews = D['data']
sentiments = D['target']

print(len(reviews), "reviews")

Keys: dict_keys(['data', 'target', 'DESCR'])
50000 reviews


This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification. This document outlines how the dataset was gathered, and how to use the files provided.
For more details see: http://ai.stanford.edu/~amaas/data/sentiment/

GPT answear when I asked what are my options for stemmers

| Method                   | Aggressiveness | Quality | Speed  | Best for                    |
| ------------------------ | -------------- | ------- | ------ | --------------------------- |
| **Porter**               | Medium         | ✔✔      | Fast   | Classic NLP                 |
| **Snowball**             | Medium         | ✔✔✔     | Fast   | Best stemmer for English    |
| **Lancaster**            | High           | ✔       | Fast   | Rare cases; very aggressive |
| **Lemmatizer (spaCy)**   | Low            | ⭐⭐⭐⭐    | Medium | Best accuracy               |
| **Lemmatizer (WordNet)** | Low            | ⭐⭐⭐     | Medium | Simpler lemmatization       |

So I opted for the "Best stemmer for english" since that what we are doing and for what I gathered Lemmatizer $ \not= $ Stemmer

In [ ]:
stemmer = SnowballStemmer("english")

def clean_review(string):
    # Remove tags HTML
    string = string.replace('<br />', ' ')  
    # Remove palavras com 20 ou mais caracteres
    string = re.sub(r'\b[a-zA-Z]{20,}\b', ' ', string)
    # Remove palavras com 3 ou mais letras repetidas consecutivamente (e.g., "yaaass", "omgggg")
    string = re.sub(r'\b\w*(.)\1{2,}\w*\b', ' ', string)
    # Filtra apenas letras
    string = re.sub(r'[^a-zA-Z]', ' ', string)
    # Remove espaços consecutivos
    string = re.sub(r'\s+', ' ', string).strip()
    # Normaliza para minúsculas
    string = string.lower()
    # Aplica Stemming
    string = " ".join(stemmer.stem(w) for w in string.split())
    return string

reviews = [clean_review(rev) for rev in reviews]

output = {"data" : reviews, "target" : sentiments}
p.dump(output,open("resources/imdbPreProcessed.p",'wb'))

In [12]:
custom_stopwords = ENGLISH_STOP_WORDS - {'no', 'not', 'nor'}

tfidVector = TfidfVectorizer(min_df=10,                    # Remove palavras que aparecem menos de 10 vezes no dataset
                        max_df=0.9,                        # Remove palavras que aparecem em 90% do dataset 
                        max_features=30000,                # Limita o maximo de features para 30.000
                        ngram_range=(1,2),                 # Utiliza unigramas e bigramas (good, very good, pretty bad)
                        token_pattern=r'\b[a-zA-Z]{2,}\b', # Ignora palavras com menos de 2 letras
                        sublinear_tf=True,                 # Term frequency passa a ter um comportamento logarítmico em vez de linear
                        stop_words='english'               # Remove stopwords em inglês
                        )

In [16]:
pre_processed_data = p.load(open("resources/imdbPreProcessed.p","rb"))
reviews = pre_processed_data['data']
sentiments = pre_processed_data['target']

tfidVector = tfidVector.fit(reviews)

tokens = tfidVector.get_feature_names_out()
X = tfidVector.transform(reviews)
X.astype(np.float32)
len(tokens)

30000

### Divisão em conjuntos de treino, teste e validação
| Treino | Teste | Validação |
|--|--|--|
|40.000 (80%)|5.000 (10%)|5.000 (10%)|

In [17]:
# Split 80% treino / 20% temporário (validação + teste)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, sentiments,
    test_size=0.20,
    random_state=42,
    stratify=sentiments
)

# Split 50% validação, 50% teste (10% / 10%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

In [18]:
param_grid = {
    'penalty': ['l2'],
    'solver': ['saga'],
    'C': [0.1, 1],
    'max_iter': [100]
}

grid_search = GridSearchCV(LogisticRegression(random_state=42, n_jobs=1), param_grid,cv=3)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}


In [19]:
best_params = {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}